In [ ]:
# All the imports we need
import numpy as np
import gtsam
from gtsam.utils.plot import plot_pose3
from matplotlib import pyplot as plt
from gtsam.symbol_shorthand import B, V, X
from gtbook.display import show
from mpl_toolkits.mplot3d import Axes3D
import time as t
from functools import partial
from error_functions import error_depth, error_dvl, error_unary_heading, error_gps


try:
  # For Google Colab
  from google.colab import output
  output.enable_custom_widget_manager()
  print(" -- Custom widgets enabled")
except:
  pass

# For interactive plots
%matplotlib widget


In [ ]:
T = 15  # The timespan of our trajectory.
dt = 1e-2  # 100 Hz frequency
velocity = np.array([2, 0, 0])  # The velocity we wish to move at.


scenarios = {
    "zero_twist": (np.zeros(3), np.zeros(3)),  # Zero motion, stationary trajectory.
    "forward_motion": (np.zeros(3), velocity),  # Move forward in the x axis at 2 m/s.
    "loop": (np.array([0., 0., -np.radians(5)]), velocity),  # A loop-de-loop trajectory.
    "sick": (np.array([np.radians(5), -np.radians(5), 0]), velocity)  # A spiral trajectory, "sick" in surfer slang.
}


def plot_scenario(scenario,
                  T,
                  dt,
                  title="IMU trajectory scenario",
                  fignum=0,
                  maxDim=5):
    for t in np.arange(0, T, dt):
        actualPose = scenario.pose(t)
        plot_pose3(fignum, actualPose, axis_length=0.3)

        translation = actualPose.translation()
        maxDim = max([max(np.abs(translation)), maxDim])
        ax = plt.gca()
        ax.set_xlim3d(-maxDim, maxDim)
        ax.set_ylim3d(-maxDim, maxDim)
        ax.set_zlim3d(-maxDim, maxDim)
        ax.set_title(title)
    

    plt.show()

In [ ]:
# IMU Parameters

accBias = np.array([-0.3, 0.1, 0.2])
gyroBias = np.array([0.1, 0.3, -0.1])

actualBias = gtsam.imuBias.ConstantBias(accBias, gyroBias)

pim_params = gtsam.PreintegrationParams.MakeSharedU(9.81)

# Some arbitrary noise sigmas
gyro_sigma = 1e-3
accel_sigma = 1e-3
I_3x3 = np.eye(3)
pim_params.setGyroscopeCovariance(gyro_sigma**2 * I_3x3)
pim_params.setAccelerometerCovariance(accel_sigma**2 * I_3x3)
pim_params.setIntegrationCovariance(1e-7**2 * I_3x3)
g = 9.81

BIAS_COVARIANCE = gtsam.noiseModel.Isotropic.Variance(6, 0.1)
pim = gtsam.PreintegratedImuMeasurements(pim_params, actualBias)



In [ ]:
# Unary stuff

# GPS 
std_gps = 6
GPS_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, std_gps) 
def get_gps_measurement(t, scenario):
    rotationNoise = gtsam.Rot3.Expmap(np.random.randn(3) * 0.1)
    translationNoise = gtsam.Point3(*np.random.randn(3) * 0.1)
    poseNoise = gtsam.Pose3(rotationNoise, translationNoise)
    gps_pose = scenario.navState(t).pose().compose(poseNoise)
    return gtsam.Point3(gps_pose.x(), gps_pose.y(), gps_pose.z())


# DVL
std_dvl_vel = np.array([0.1, 0.1, 0.1])
DVL_NOISE = gtsam.noiseModel.Diagonal.Sigmas(std_dvl_vel)
def get_dvl_measurement(t, scenario):
    velocity = scenario.navState(t).velocity() + np.random.randn(3) * 0.1
    return velocity


# Heading
std_orientation = np.deg2rad(3)
HEADING_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, std_orientation)
def get_orientation_measurement(t, scenario):
    rotationNoise = gtsam.Rot3.Expmap(np.random.randn(3) * 0.1)
    translationNoise = gtsam.Point3(*np.random.randn(3) * 1)
    poseNoise = gtsam.Pose3(rotationNoise, translationNoise)
    orientation_matrix = scenario.navState(t).pose().compose(poseNoise).rotation()
    return orientation_matrix



# Depth
std_depth = 0.1
DEPTH_NOISE = gtsam.noiseModel.Isotropic.Sigma(1, std_depth)
def get_depth_measurement(t, scenario):
    rotationNoise = gtsam.Rot3.Expmap(np.random.randn(3) * 0.1)
    translationNoise = gtsam.Point3(*np.random.randn(3) * 0.1)
    poseNoise = gtsam.Pose3(rotationNoise, translationNoise)
    gps_pose = scenario.navState(t).pose().compose(poseNoise)
    depth_meas = np.array([gps_pose.z()])
    return depth_meas






In [ ]:
def init_incremental_fixed_lag_smoother():
    # Incremental Fixed Lag Smoother Params
    isam_params = gtsam.ISAM2Params()
    isam_params.setRelinearizeThreshold(0.1)
    isam_params.relinearizeSkip = 1

    # important to set to true
    isam_params.findUnusedFactorSlots = True
    lag = 5.0 # the sliding window time frame    

    # key value to time_stamp
    time_stamp_map = gtsam.FixedLagSmootherKeyTimestampMap()

    return gtsam.IncrementalFixedLagSmoother(lag, isam_params), time_stamp_map





In [ ]:
def add_priors(scenario, graph, initial, time_stamps, smoother):
    # Noise models for


    priorNoise = gtsam.noiseModel.Isotropic.Sigma(6, 0.1)
    velNoise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)

    
    initial_state = scenario.navState(0)
    graph.push_back(
        gtsam.PriorFactorPose3(X(0), initial_state.pose(), priorNoise))
    
    graph.push_back(
        gtsam.PriorFactorVector(V(0), initial_state.velocity(), velNoise))

    graph.push_back(gtsam.PriorFactorConstantBias(B(0), actualBias, gtsam.noiseModel.Isotropic.Sigma(6, 0.1)))
    
    


    initial.insert(B(0), actualBias)
    initial.insert(X(0), initial_state.pose())
    initial.insert(V(0), initial_state.velocity())

    # fill time_stamp map
    time = 0.
    time_stamps[X(0)] = time
    time_stamps[V(0)] = time
    time_stamps[B(0)] = time
    

    smoother.update(graph, initial, time_stamps)
    result = smoother.calculateEstimate()
    # init things after update
    graph = gtsam.NonlinearFactorGraph()
    time_stamps = gtsam.FixedLagSmootherKeyTimestampMap()
    initial.clear()

    
    return graph, initial, smoother, result, initial_state, time_stamps





In [ ]:
def main_loop(runner, scenario, graph, initial, pim, time_stamps, smoother, T):
    # The factor index for the estimation rate
    i = 0
    FACTOR_GRAPH_PERIOD = 1 # in seconds
    GPS_UNARY_PERIOD = 5 

    # Below unary periods in reality will be added every tick of the factor graph, so they all equal one another
    DVL_UNARY_PERIOD = FACTOR_GRAPH_PERIOD 
    HEADING_UNARY_PERIOD = FACTOR_GRAPH_PERIOD 
    DEPTH_UNARY_PERIOD = FACTOR_GRAPH_PERIOD 
    bias_key = B(0)

    graph, initial, smoother, result, initial_state, time_stamps = add_priors(scenario, graph, initial, time_stamps, smoother)
    prev_state = initial_state


    for k, t in enumerate(np.arange(0, T, dt)):
        # get measurements and add them to PIM

        measuredOmega = runner.measuredAngularVelocity(t)
        measuredAcc = runner.measuredSpecificForce(t)

        ### This is where all the magic happens!
        pim.integrateMeasurement(measuredAcc, measuredOmega, dt)

       
        if (k + 1) % int((1 / dt) * FACTOR_GRAPH_PERIOD) == 0:
            time = t + dt
            
            if i == 0:
                

                predicted_state_imu_dead_reckoning = pim.predict(prev_state, actualBias)
                initial.insert(X(1), predicted_state_imu_dead_reckoning.pose())
                initial.insert(V(1), predicted_state_imu_dead_reckoning.velocity())
                time_stamps[X(1)] = time
                time_stamps[V(1)] = time
                prev_state = predicted_state_imu_dead_reckoning
                # print('adding index 1')
            elif i >= 2:
                # print(f'adding index {i}')
                predicted_state_imu_dead_reckoning = pim.predict(prev_state, actualBias)
                initial.insert(X(i), predicted_state_imu_dead_reckoning.pose())
                initial.insert(V(i), predicted_state_imu_dead_reckoning.velocity())
                time_stamps[X(i)] = time
                time_stamps[V(i)] = time
                prev_state = predicted_state_imu_dead_reckoning
                

            if i > 0:
                # print(f'time @ index {i}: ', t + dt)

                if i % 2 == 0:
                    bias_key += 1
                    initial.insert(bias_key, actualBias)
                    # note that gtsam.imuBias.ConstantBias() defaults to zeroes
                    bias_between_factor = gtsam.BetweenFactorConstantBias(bias_key - 1, bias_key, gtsam.imuBias.ConstantBias(), BIAS_COVARIANCE)
                    graph.add(bias_between_factor)
                    time_stamps[bias_key] = time

                
                # Create IMU factor every second.

                # print('adding imu factor')
                factor = gtsam.ImuFactor(X(i - 1), V(i - 1), X(i), V(i), bias_key, pim)
                graph.push_back(factor)

                # We have created the binary constraint, so we clear out the preintegration values.
                pim.resetIntegration()

                
                # add unaries
                
                # GPS every so often, constrain pose x y z estimate
                if i % GPS_UNARY_PERIOD == 0:
                    graph.add(gtsam.CustomFactor(GPS_NOISE, [X(i)], partial(error_gps, get_gps_measurement(t, scenario))))
                
                # Heading every tick of the factor graph (still you may configure the period), constrains orientation
                if i % HEADING_UNARY_PERIOD == 0:
                    graph.add(gtsam.CustomFactor(HEADING_NOISE, [X(i)],partial(error_unary_heading, [get_orientation_measurement(t, scenario)])))
                
                # Depth every tick of the factor graph, constrains z component of pose estimate
                if i % DEPTH_UNARY_PERIOD == 0:
                    graph.add(gtsam.CustomFactor(DEPTH_NOISE, [X(i)], partial(error_depth, [get_depth_measurement(t, scenario)])))

                # Constrains velocity x y z
                if i % DVL_UNARY_PERIOD == 0:
                    graph.add(gtsam.CustomFactor(DVL_NOISE, [V(i)], partial(error_dvl, get_dvl_measurement(t, scenario)) ))



            i += 1

            # if i > 1:
            # print('About to update')
            smoother.update(graph, initial, time_stamps)
            result = smoother.calculateEstimate()
            # print('\n\nBEGINNING\n\n',result,'\n\n\END\n\n')
            graph = gtsam.NonlinearFactorGraph()
            time_stamps = gtsam.FixedLagSmootherKeyTimestampMap()
            initial.clear()

            

    return graph, result

In [ ]:
def plot_trajectory(values: gtsam.Values,
                    title: str = "Estimated Trajectory",
                    fignum: int = 1,
                    show: bool = False):
    


    i = 0
    while values.exists(X(i)) == False:
        i += 1


    while values.exists(X(i)):
        pose_i = values.atPose3(X(i))
        plot_pose3(fignum, pose_i, 1)
        i += 1
    plt.title(title)

    gtsam.utils.plot.set_axes_equal(fignum)

    plt.ioff()

    if show:
        plt.show()

In [ ]:

scenario = gtsam.ConstantTwistScenario(*scenarios["forward_motion"])
plot_scenario(scenario, T, dt, title="Forward Motion", fignum=5)
runner = gtsam.ScenarioRunner(scenario, pim_params, dt, actualBias)
# graph, initial, isam, result = add_priors(scenario, graph, initial, isam)

graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()
smoother, time_stamps = init_incremental_fixed_lag_smoother()
graph, result = main_loop(runner, scenario, graph, initial,pim, time_stamps, smoother, T)
# print(result)

plot_trajectory(result, title="ISAM Estimate", fignum=7, show=True)

In [ ]:
pim = gtsam.PreintegratedImuMeasurements(pim_params, actualBias)
scenario = gtsam.ConstantTwistScenario(*scenarios["zero_twist"])
plot_scenario(scenario, T, dt, title="Zero Twist", fignum=8)

runner = gtsam.ScenarioRunner(scenario, pim_params, dt, actualBias)


graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()
isam = gtsam.ISAM2()

smoother, time_stamps = init_incremental_fixed_lag_smoother()
graph, result = main_loop(runner, scenario, graph, initial,pim, time_stamps, smoother, T)
plot_trajectory(result, title="ISAM Estimate", fignum=9, show=True)



In [ ]:
pim = gtsam.PreintegratedImuMeasurements(pim_params, actualBias)

scenario = gtsam.ConstantTwistScenario(*scenarios["loop"])
plot_scenario(scenario, T, dt, title="Loop", fignum=10)

runner = gtsam.ScenarioRunner(scenario, pim_params, dt, actualBias)


graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()
isam = gtsam.ISAM2()

# graph, initial, isam, result = add_priors(scenario, graph, initial, isam)
smoother, time_stamps = init_incremental_fixed_lag_smoother()
graph, result = main_loop(runner, scenario, graph, initial,pim, time_stamps, smoother, T)
plot_trajectory(result, title="ISAM Estimate", fignum=11, show=True)

In [ ]:
pim = gtsam.PreintegratedImuMeasurements(pim_params, actualBias)
scenario = gtsam.ConstantTwistScenario(*scenarios["sick"])
plot_scenario(scenario, T, dt, title="Sick", fignum=12)

runner = gtsam.ScenarioRunner(scenario, pim_params, dt, actualBias)


graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()
isam = gtsam.ISAM2()

# graph, initial, isam, result = add_priors(scenario, graph, initial, isam)
smoother, time_stamps = init_incremental_fixed_lag_smoother()
graph, result = main_loop(runner, scenario, graph, initial,pim, time_stamps, smoother, T)
plot_trajectory(result, title="ISAM Estimate", fignum=13, show=True)